Estudiar correlación directa entre proteínas y metabolitos en las tres cohortes y ver si se valida, especialmente entre PESA-V1 y AWHS

In [1]:
import pandas as pd
from dotmap import DotMap
import pickle
import sys
import numpy as np
import re

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

utilsPath = r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\utils'
if utilsPath not in sys.path:
    sys.path.append(utilsPath)

In [2]:
q2i = {
    'v1': pd.read_csv(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Proteomics\PESA\WorkingFiles\q2info.tsv', sep='\t'),
    'v2': pd.read_csv(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Proteomics\PESA_V2\WorkingFiles\q2info.tsv', sep='\t'),
    'aw2': pd.read_csv(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Proteomics\AWHS\WorkingFiles\q2info.tsv', sep='\t'),
    'aw1': pd.read_csv(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Proteomics\AWHS\WorkingFiles\q2info.tsv', sep='\t')
}

m2i = {
    'v1': pd.read_csv(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\PESA\WorkingFiles\f2i.tsv', sep='\t'),
    'v2': pd.read_csv(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\PESA_V2\WorkingFiles\f2i.tsv', sep='\t'),
    'aw2': pd.read_csv(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\AWHS\WorkingFiles\m2info.tsv', sep='\t'),
    'aw1': pd.read_csv(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\AWHS\WorkingFiles\m2info.tsv', sep='\t')
}

In [3]:
corr = {}

with open(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Analysis\02-Correlations\PESA\corr.pkl', 'rb') as f:
    corr['v1'], corrSL = pickle.load(f)
with open(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Analysis\02-Correlations\PESA_V2\corr.pkl', 'rb') as f:
    corr['v2'], corrSL = pickle.load(f)
with open(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Analysis\02-Correlations\AWHS\corr.pkl', 'rb') as f:
    corr['aw2'], corrSL = pickle.load(f)
with open(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Analysis\02-Correlations\AWHS_MS1\corr.pkl', 'rb') as f:
    corr['aw1'], corrSL = pickle.load(f)

In [4]:
adpv = {}

with open(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Analysis\02-Correlations\PESA\pvals.pkl', 'rb') as f:
    pv, adpv['v1'] = pickle.load(f)
with open(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Analysis\02-Correlations\PESA_V2\pvals.pkl', 'rb') as f:
    pv, adpv['v2'] = pickle.load(f)
with open(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Analysis\02-Correlations\AWHS\pvals.pkl', 'rb') as f:
    pv, adpv['aw2'] = pickle.load(f)
with open(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Analysis\02-Correlations\AWHS_MS1\pvals.pkl', 'rb') as f:
    pv, adpv['aw1'] = pickle.load(f)

In [5]:
rcca = {}
for i in corr:
    rcca[i] = corr[i].rcca.qm.a

rccaPV = {}
for i in adpv:
    rccaPV[i] = adpv[i].rcca.qm.a

In [6]:
i = 'v1'
rdf = {}
for i in rcca:
    rdf[i] = pd.merge(

        pd.melt(
            rcca[i].reset_index(),
            id_vars='index',
            value_vars=rcca[i].columns
        ).rename(columns={'index':'q', 'variable':'m', 'value':'rcca'}),

        pd.melt(
            rccaPV[i].reset_index(),
            id_vars='index',
            value_vars=rccaPV[i].columns
        ).rename(columns={'index':'q', 'variable':'m', 'value':'adpv'}),

        on = ['q', 'm'],
        how='inner'

    )

    rdf[i]['q'] = [j.replace('.','-') for j in rdf[i]['q']]

In [7]:
i='aw'

for i in rdf:
    rdf[i] = pd.merge(
        rdf[i],
        q2i[i].rename(columns={'fid':'q'}),
        on='q', how='left'
    )

In [8]:
i = 'v1'

for i in rdf:
    rdf[i] = pd.merge(
        rdf[i],
        m2i[i].rename(columns={'fid':'m'}),
        on='m', how='left'
    )

Figura Paper - Correlaciones

In [9]:
from itertools import product

In [54]:
# Lipids filtered by Alessia

flipids = {
    'v1': pd.read_excel('PC_PE_AA_DGLA_MSMS.xlsx', sheet_name='v1', index_col=0),
    'v2': pd.read_excel('PC_PE_AA_DGLA_MSMS.xlsx', sheet_name='v2', index_col=0),
    'aw1': pd.read_excel('PC_PE_AA_DGLA_MSMS.xlsx', sheet_name='aw', index_col=0),
    'aw2': pd.read_excel('PC_PE_AA_DGLA_MSMS.xlsx', sheet_name='aw', index_col=0)
}


In [21]:
# lipids = [
#     (i, j[0]+k[0], j[1]+k[1])
#     for i,j,k in 
#     # list(product(['PE', 'PC', 'LPE', 'LPC'], [(20,4), (20,3)], [(18,0), (18,1), (18,2), (16, 0), (16, 1), (0,0)]))
#     list(product(['PE', 'PC'], [(20,4), (20,3)], [(18,0), (18,1), (18,2), (16, 0), (16, 1), (0,0)]))
#  ]

In [13]:
proteins = [
    'P00738', # HPT
    'P05546', # HEP2
    'P01833', # PIGR
    'P01877', # IGHA2
]

proteinsName = ['HPT', 'HEP2','PIGR', 'IGHA2']

In [14]:
import plotly.express as px

palette = px.colors.qualitative.Plotly

In [141]:
ch = 'aw1'
for ch, chName in [('v1', 'PESA-V1'), ('v2', 'PESA-V2'), ('aw1', 'AWHS-MS1'), ('aw2', 'AWHS-MS2')]:
    fig = make_subplots(rows=1, cols=len(proteins), subplot_titles=proteinsName)
    for n,i in enumerate(proteins):

        tmp = rdf[ch][rdf[ch].q == i].sort_values('rcca')
        if tmp.shape[0] ==0: continue
        tmp['Y'] = np.arange(0,1,1/tmp.shape[0])


        fig.add_trace(go.Scatter(
            x=tmp.rcca, y=tmp['Y'],  name='All lipids', line_width=1, line_color='rgb(120,120,120)',#palette[0],
            showlegend=True if n==0 else False
        ), row=1, col=n+1)

        for m,k in enumerate(['PC', 'PE']):#, 'LPC', 'LPE']):
            tmp2 = pd.merge(
                tmp.drop('ID', axis=1) if 'ID' in tmp.columns else tmp,
                flipids[ch].loc[:, ['ID']].reset_index(names='m'),
                on='m',
                how='inner'
            )

            tmp2 = tmp2[[k in kk for kk in tmp2.ID]]
            tmp2['Y2'] = np.arange(0,1,1/tmp2.shape[0])

            fig.add_trace(go.Scatter(
                x=tmp2.rcca, y=tmp2['Y'], mode='markers', marker_size=5,
                text=tmp2.m,
                textposition='bottom center', marker_color=palette[m+1],
                showlegend=True if n==0 else True, name=k, legendgroup=m
            ), row=1, col=n+1)



    fig.update_xaxes(title='r (rCCA)')
    fig.update_yaxes(title='Rank/N', col=1)

    fig.update_layout(title=f'{chName.upper()}')
    fig.show()

: 

Volcano Plot

In [72]:
xm = {
    'v1': pd.read_csv(r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\PESA\WorkingFiles\Xm_norm.tsv", sep='\t', index_col='Seqn'),
    'v2': pd.read_csv(r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\PESA_V2\WorkingFiles\Xm_norm.tsv", sep='\t', index_col='Seqn'),
    'aw2': pd.read_csv(r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\AWHS\WorkingFiles\Xm_norm_MS2.tsv", sep='\t', index_col='fid'),
    'aw1': pd.read_csv(r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\AWHS\WorkingFiles\Xm_norm_MS1.tsv", sep='\t', index_col='fid')
}

In [73]:
xq = {
    'v1': pd.read_csv(r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Proteomics\PESA\WorkingFiles\Xq_minus_X_norm.tsv", sep='\t', index_col='Seqn'),
    'v2': pd.read_csv(r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Proteomics\PESA_V2\WorkingFiles\Xq_minus_X_norm.tsv", sep='\t', index_col='seqn'),
    'aw2': pd.read_csv(r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Proteomics\AWHS\WorkingFiles\Xq_minus_X_norm.tsv", sep='\t', index_col='Seqn'),
    'aw1': pd.read_csv(r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Proteomics\AWHS\WorkingFiles\Xq_minus_X_norm.tsv", sep='\t', index_col='Seqn')
}

In [74]:
mdata = {
    'v1': pd.read_csv(r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metadata\PESA\WorkingFiles\main_metadata.tsv",sep='\t', index_col='Seqn'),
    'v2': pd.read_csv(r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metadata\PESA_V2\WorkingFiles\main_metadata.tsv",sep='\t', index_col='Seqn'),
    'aw2': pd.read_csv(r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metadata\AWHS\WorkingFiles\main_metadata.tsv",sep='\t', index_col='Seqn'),
    'aw1': pd.read_csv(r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metadata\AWHS\WorkingFiles\main_metadata.tsv",sep='\t', index_col='Seqn')
}

for i in mdata:
    mdata[i] = mdata[i].loc[xm[i].index]


In [75]:
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import fdrcorrection

In [76]:
m2i['aw2'].loc[:, 'TP_ID'] = m2i['aw2']['NAME']
m2i['aw1'].loc[:, 'TP_ID'] = m2i['aw1']['NAME']

In [77]:
# REMOVE TAG
m2i['aw2'] = m2i['aw2'][m2i['aw2'].CLASS != 'TAG']
xm['aw2'] = xm['aw2'][np.intersect1d(xm['aw2'].columns, m2i['aw2'].fid)]

m2i['aw1'] = m2i['aw1'][m2i['aw1'].CLASS != 'TAG']
xm['aw1'] = xm['aw1'][np.intersect1d(xm['aw1'].columns, m2i['aw1'].fid)]

In [78]:
i = 'v1'

vp = {}

for i in xm:

    vp[i] = pd.DataFrame()

    vp[i]['lfc'] = xm[i].loc[mdata[i].index[mdata[i].Group=='D']].mean()-xm[i].loc[mdata[i].index[mdata[i].Group=='C']].mean()

    vp[i]['t'], vp[i]['pv'] = ttest_ind(
        xm[i].loc[mdata[i].index[mdata[i].Group=='D']],
        xm[i].loc[mdata[i].index[mdata[i].Group=='C']]
    )

    vp[i]['fdr'] = fdrcorrection(vp[i].pv)[1]

In [97]:
i = 'v1'

pd.isna(vp[i].Alessia[0])

True

In [90]:
i = 'v1'

for i in vp:
    vp[i] = vp[i].join(flipids[i][['ID']].rename(columns={'ID': 'Alessia'}))

In [138]:
i = 'v1'

fig = make_subplots(rows=1, cols=4, shared_yaxes=True, subplot_titles=['PESA-V1', 'PESA-V2', 'AWHS-MS1', 'AWHS-MS2'])

n = 0

for n,i in enumerate(['v1', 'v2', 'aw1', 'aw2']):#enumerate(vp):

    fig.add_trace(go.Scatter(
        #x=vp[i].lfc, y=-np.log10(vp[i].pv),
        y=vp[i].lfc, x=np.random.uniform(-0.2,0.2,vp[i].shape[0]),
        mode='markers', marker_size=1, marker_color=palette[0], showlegend=False
    ), row=1, col=n+1)

    for m,k in enumerate(['PC', 'PE']):#, 'LPC', 'LPE']):
        fig.add_trace(go.Scatter(
            #x=vp[i].loc[[k in j for j in vp[i].label], 'lfc'], y=-np.log10(vp[i].loc[[k in j for j in vp[i].label], 'pv']),
            x=np.random.uniform((m+1)-0.1,m+1+0.1,vp[i].shape[0]),
            y=vp[i][[not pd.isna(j) and k in j for j in vp[i].Alessia]]['lfc'], 
            mode='markers', marker_size=3, marker_color=palette[m+1], name=k, showlegend=False
        ), row=1, col=n+1)

fig.update_xaxes(tickvals=[0,1,2,3,4], ticktext=['All', 'PC', 'PE', 'LPC', 'LPE'])
# fig.update_yaxes(title='LogFC')
# fig.update_layout(width=20)
fig.show()

In [25]:
i = vp['v1']

In [26]:
i = 'v1'

with pd.ExcelWriter(f"PC_PE_AA_DGLA_output.xlsx") as writer:

    for i in m2i:
        out = m2i[i].set_index('fid').loc[xm[i].columns, :]
        out[out.label!=''].to_excel(writer, sheet_name=i, index=True)

Regresión Logística de cada PE con HEP2

In [111]:
mdata['v1']['FHS10y'] = mdata['v1']['Framingham 10y']
mdata['v2']['FHS10y'] = mdata['v2']['Framingham 10y']

In [112]:
import statsmodels.api as sm

In [115]:
mdata['v1']['CACS'] = mdata['v1']['Calcium_Score'] # 'Calcium_Score'
mdata['v2']['CACS'] = mdata['v2']['Calcium_Score'] # 'Calcium_Score'
mdata['aw1']['CACS'] = mdata['aw1']['Calcio Score'] # 'Calcio Score'
mdata['aw2']['CACS'] = mdata['aw2']['Calcio Score'] # 'Calcio Score'


In [116]:
for i in mdata:
    mdata[i]['CACS_bin'] = [1 if j>0 else 0 for j in mdata[i].CACS]

In [123]:
vp[ch].loc[[not pd.isna(i) and 'PE' in i for i in vp[ch].Alessia]].index.tolist()

['C18P116', 'C18P168', 'C18P171', 'C18P182', 'C18N53', 'C18N71']

In [124]:
res = {}

mym = {}

myq = 'P05546'
predictCol = 'CACS_bin'

for ch in ['v1', 'v2', 'aw2', 'aw1']:
    res[ch] = {'m':{},'fhs':{},'hep':{},'fhs+hep':{},'allm':{}}

    mym[ch] = vp[ch].loc[[not pd.isna(i) and 'PE' in i for i in vp[ch].Alessia]].index.tolist()

    data = mdata[ch][[predictCol, 'FHS10y']].join(xq[ch][[myq]], how='inner').join(xm[ch][mym[ch]], how='inner')


    res[ch]['m'] = {
        i: sm.Logit(data[[predictCol]], sm.add_constant(data[[i]])).fit()
        for i in mym[ch]
    }
    res[ch]['allm']['m'] = sm.Logit(data[[predictCol]], sm.add_constant(data[mym[ch]].T.drop_duplicates().T)).fit()

    res[ch]['fhs'] = {
        i: sm.Logit(data[[predictCol]], sm.add_constant(data[['FHS10y',i]])).fit()
        for i in mym[ch]
    }
    res[ch]['allm']['fhs'] = sm.Logit(data[[predictCol]], sm.add_constant(data[['FHS10y',*mym[ch]]].T.drop_duplicates().T)).fit()

    res[ch]['hep'] = {
        i: sm.Logit(data[[predictCol]], sm.add_constant(data[[myq,i]])).fit()
        for i in mym[ch]
    }
    res[ch]['allm']['hep'] = sm.Logit(data[[predictCol]], sm.add_constant(data[[myq,*mym[ch]]].T.drop_duplicates().T)).fit()

    res[ch]['fhs+hep'] = {
        i: sm.Logit(data[[predictCol]], sm.add_constant(data[['FHS10y', myq,i]])).fit()
        for i in mym[ch]
    }
    res[ch]['allm']['fhs+hep'] = sm.Logit(data[[predictCol]], sm.add_constant(data[['FHS10y', myq,*mym[ch]]].T.drop_duplicates().T)).fit()


Optimization terminated successfully.
         Current function value: 0.676511
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.680132
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.679762
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.675788
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.660289
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.662085
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.655900
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.653204
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.654498
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.654293
  

In [125]:
summary = {}
ch = 'v1'

for ch in ['v1', 'v2', 'aw2', 'aw1']:
    summary[ch] = {}
    summary[ch]['m'] = pd.DataFrame([(key, value.params[key], value.pvalues[key]) for key,value in res[ch]['m'].items()], columns=['fid', 'PE coef', 'PE pv'])
    summary[ch]['hep'] = pd.DataFrame([(key,value.params[myq], value.pvalues[myq], value.params[key], value.pvalues[key]) for key,value in res[ch]['hep'].items()], columns=['fid', 'HEP2 coef', 'HEP2 pv', 'PE coef', 'PE pv'])
    summary[ch]['fhs'] = pd.DataFrame([(key,value.params['FHS10y'], value.pvalues['FHS10y'], value.params[key], value.pvalues[key]) for key,value in res[ch]['fhs'].items()], columns=['fid', 'FHS10y coef', 'FHS10y pv', 'PE coef', 'PE pv'])
    summary[ch]['fhs+hep'] = pd.DataFrame([(key,value.params['FHS10y'], value.pvalues['FHS10y'],value.params[myq], value.pvalues[myq], value.params[key], value.pvalues[key]) for key,value in res[ch]['fhs+hep'].items()], 
                columns=['fid', 'FHS10y coef', 'FHS10y pv', 'HEP2 coef', 'HEP2 pv', 'PE coef', 'PE pv'])


In [126]:
for ch in summary:
    for i in summary[ch]:
        summary[ch][i] = summary[ch][i].set_index('fid')
        summary[ch][i].columns = pd.MultiIndex.from_tuples([(i,j) for j in summary[ch][i].columns])


In [127]:
for ch in summary:
    summary[ch] = summary[ch]['m'].join(
        summary[ch]['hep']
    ).join(
        summary[ch]['fhs']
    ).join(
        summary[ch]['fhs+hep']
    )

In [128]:
from functools import reduce

In [129]:
i = 'v2'
j = 'fhs+hep'

summary2 = {}

for i in res:
    summary2[i] = reduce(
        lambda x,y: x.join(y, how='outer'),
        [
            pd.DataFrame({
                (j, 'coef'):{i:j for i,j in res[i]['allm'][j].params.items() if i!='const'},
                (j, 'pvalue'):{i:j for i,j in res[i]['allm'][j].pvalues.items() if i!='const'}
            })
            for j in res[i]['allm']
        ]
    )

In [130]:
summary
m2i['v1']
ch = 'v1'
with pd.ExcelWriter(f"LogisticRegression_{predictCol.replace('/','-')}.xlsx") as writer:

    for ch in summary:
        out = m2i[ch].set_index('fid').loc[summary[ch].index, :]
        out.to_excel(writer, sheet_name=f'{ch}-INFO', index=True)
        out = summary[ch]
        out.to_excel(writer, sheet_name=f'{ch}-LR', index=True)
        summary2[ch].to_excel(writer, sheet_name=f'{ch}-LR-ALLm', index=True)
        

In [38]:
ch = 'aw'

mcand = summary[ch].sort_values(('m', 'PE pv')).index.tolist()
data = mdata[ch][[predictCol]].join(xm[ch][mcand], how='inner')

selected = [mcand[0]]
metrics = {'bic':[], 'aic':[], 'pvalue':[], 'features':[]}

model = sm.Logit(data[[predictCol]], sm.add_constant(data[selected].T.drop_duplicates().T)).fit()
metrics['bic'], metrics['aic'], metrics['pvalue'], metrics['features'] = [model.bic], [model.aic], [model.llr_pvalue], [selected]

for f in mcand[1:]:
    model = sm.Logit(data[[predictCol]], sm.add_constant(data[[*selected, f]].T.drop_duplicates().T)).fit()
    print(model.bic, metrics['bic'][-1])
    if model.bic<metrics['aic'][-1]:
        selected.append(f)
        metrics['bic'].append(model.bic), metrics['aic'].append(model.aic), metrics['pvalue'].append(model.llr_pvalue), metrics['features'].append(selected)


Optimization terminated successfully.
         Current function value: 0.618124
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.617821
         Iterations 5
447.5602522019329 441.9190490394998
Optimization terminated successfully.
         Current function value: 0.618068
         Iterations 5
447.7316647088103 441.9190490394998
Optimization terminated successfully.
         Current function value: 0.618110
         Iterations 5
447.76123495796696 441.9190490394998
Optimization terminated successfully.
         Current function value: 0.617675
         Iterations 5
447.4584009230023 441.9190490394998
Optimization terminated successfully.
         Current function value: 0.618084
         Iterations 5
447.7430018223841 441.9190490394998
Optimization terminated successfully.
         Current function value: 0.618124
         Iterations 5
447.77066299847144 441.9190490394998
Optimization terminated successfully.
         Current function valu

In [39]:
metrics

{'bic': [441.9190490394998],
 'aic': [434.21464407995086],
 'pvalue': [0.11716118921146787],
 'features': [['N14']]}

In [40]:
model.llr_pvalue

0.28649399046708796

In [41]:
i = 'aw1'

dir(res[i]['allm']['m'])
res[i]['allm']['m'].summary2()
# res[i]['m']['N13'].summary2()#['m'].summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                        Results: Logit
===============================================================
Model:              Logit            Pseudo R-squared: 0.037   
Dependent Variable: CACS_bin         AIC:              396.3566
Date:               2023-08-30 14:52 BIC:              433.7545
No. Observations:   311              Log-Likelihood:   -188.18 
Df Model:           9                LL-Null:          -195.32 
Df Residuals:       301              LLR p-value:      0.11283 
Converged:          1.0000           Scale:            1.0000  
No. Iterations:     6.0000                                     
-----------------------------------------------------------------
         Coef.    Std.Err.      z      P>|z|     [0.025    0.975]
-----------------------------------------------------------------
const   -0.7788     0.1280   -6.0835   0.0000   -1.0298   -0.5279
N10     -0.7543     0.4336   -1.7397   0.0819   -1.6040    0.0955
N13      0.9021     0.4114    2.1927   0.0283    0.0957    1.7084
N14      0.0126     0.3503    0.0359   0.9714   -0.6740    0.6992
N15      0.1687     0.2252    0.7492   0.4538   -0.2727    0.6101
P146     0.1451     0.1533    0.9471   0.3436   -0.1552    0.4455
P15      0.7699     0.5081    1.5152   0.1297   -0.2260    1.7659
P21     -0.4578     0.4030   -1.1358   0.2560   -1.2477    0.3322
P508    -0.1812     0.1417   -1.2787   0.2010   -0.4589    0.0965
P9      -0.3877     0.2751   -1.4093   0.1588   -0.9269    0.1515
===============================================================

"""